In [25]:
import pandas as pd
train_df = pd.read_json (r'/home/areej/anaconda3/AAAI-21-SDU-shared-task-2-AD-master/dataset/train.json')
#print (df.head(5))
dev_df = pd.read_json (r'/home/areej/anaconda3/AAAI-21-SDU-shared-task-2-AD-master/dataset/dev.json')
#print (df.head(5))

In [46]:
CNN_df=pd.DataFrame()
for index,row in train_df.iterrows():
    if row['tokens'][row['acronym']]=='ART':
        CNN_df=CNN_df.append(row)

        
print(CNN_df.shape)

(200, 4)


In [44]:
import nltk
import string
import re
import csv
import spacy
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, average_precision_score
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import  confusion_matrix
from sklearn.utils import shuffle
import os
from multiprocessing import Process, current_process
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [36]:
#get both side for acronymn

term=train_df['tokens'][0]
left_side=term[0:int(train_df['acronym'][0])]
right_side=term[int(train_df['acronym'][0])+1:]
print(term)

print(term[0:int(train_df['acronym'][0])])
print(term[int(train_df['acronym'][0])+1:])
print(term[train_df['acronym'][0]])

['In', 'summary', ',', 'it', 'is', 'evident', 'that', 'their', 'complexities', 'are', 'in', 'increasing', 'order', ':', 'leakage', '-', 'based', ',', 'Max', '-', 'SR', ',', 'and', 'generalized', 'EDAS', '.']
['In', 'summary', ',', 'it', 'is', 'evident', 'that', 'their', 'complexities', 'are', 'in', 'increasing', 'order', ':', 'leakage', '-', 'based', ',', 'Max', '-']
[',', 'and', 'generalized', 'EDAS', '.']
SR


In [8]:
def check_special(wordSpecial):
    t= False
    if (bool(re.search('^[a-zA-Z0-9]*$',wordSpecial))==True) and wordSpecial.isdigit()==False:
        t=True
    return t

In [7]:
import nltk
import string
import re
import csv
import spacy
import pandas as pd
import numpy as np
import json

In [44]:
import json
with open('/home/ArMuas/anaconda3/AAAI-21-SDU-shared-task-2-AD-master/dataset/diction.json') as file:
    diction = json.load(file)
    
#sense vetor generation
'''
for acr, lfs in diction.items():
        for lf in lfs:
            print(acr,lf)
            print(get_max_embedding(lf))
            #freq[lf]=get_max_embedding(lf)
'''
print(diction['ADA'])          
            

american diabetes association 's


In [20]:
import re
def get_max_embedding(sense):
    emb_vector=np.zeros(300)
    for word in sense:
        if check_special(word)==True:
            embedding_vector = np.nan_to_num(model[word.lower()])
            if embedding_vector is not None:
                emb_vector = np.maximum(emb_vector,embedding_vector)
    
    return emb_vector

In [28]:
def train_test_data(feature_list,sense):
    v = DictVectorizer(sparse=None)
    X=feature_list
    #X = v.fit_transform(feature_list)
    Encoder = LabelEncoder()
    X_train, X_test, y_train, y_test = train_test_split(X, sense, test_size = 0.2, random_state=0)
    Train_Y = Encoder.fit_transform(y_train)
    Test_Y = Encoder.fit_transform(y_test)
    #print(len(X_train ),len(y_train))
    return X_train, X_test, Train_Y, Test_Y

In [47]:
models = []
#models.append(('LR', LogisticRegression(max_iter=1000)))
#models.append(('KNN', KNeighborsClassifier(n_neighbors=5)))
#models.append(('CART', DecisionTreeClassifier(random_state=0)))
models.append(('NB', GaussianNB()))
models.append(('SVM', svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto' )))
with open('/home/areej/anaconda3/AAAI-21-SDU-shared-task-2-AD-master/dataset/diction.json') as file:
    diction = json.load(file)
    
with open('/home/areej/anaconda3/AAAI-21-SDU-shared-task-2-AD-master/dataset/train.json') as file:
    data = json.load(file)
embedding_matrix = np.zeros((1, 300)) #initalize embedding matrix
'''
predictions = []
for d in data:
    pred = {
        'id': d['id'],
        'prediction': ''}
    
    candids = diction[d['tokens'][d['acronym']]]
    
    #sense embedding
    sens_emd_dic={}
    for sense in candids:
        sens_emd_dic[sense]=get_max_embedding(sense.split())
'''
for index,row in CNN_df.iterrows():
        
    #context embedding for each senstence
       
    if row['acronym'] != 0:        
        left_side=row['tokens'][0:int(row['acronym'])]
    else:
        left_side=[]
    
    right_side=row['tokens'][int(row['acronym'])+1:]
    
    context_vec=(np.nan_to_num(get_max_embedding(right_side))
                                +
                                np.nan_to_num(get_max_embedding(left_side))/2)
    embedding_matrix = np.vstack([embedding_matrix, context_vec]) #add vector features

x = np.delete(embedding_matrix, (0), axis=0) #delete first row

senses=CNN_df['expansion']

X_train, X_test, Train_Y, Test_Y= train_test_data(x,senses)
    
    
for name, mod in models:
    mod.fit(X_train,Train_Y)
    prediction=mod.predict(X_test)
    print(name,metrics.accuracy_score(Test_Y,prediction)*100)
    
'''
            
    pred['prediction'] = pred_sense
    predictions.append(pred)        
    #print(d['id'],d['expansion'],max_,pred )
'''
            


/home/areej/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


NB 100.0
SVM 100.0


"\n            \n    pred['prediction'] = pred_sense\n    predictions.append(pred)        \n    #print(d['id'],d['expansion'],max_,pred )\n"

In [76]:
#mod.classes_
#labels = np.argmax(Test_Y, axis=1)
classes = mod.classes_
#labels = [classes[i] for i in labels]
Encoder = LabelEncoder()
predictions_test = Encoder.inverse_transform(prediction)

NotFittedError: This LabelEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [22]:
with open('output_train_cosine_FastText_MAX.json', 'w') as file:
    json.dump(predictions, file)   

In [31]:
print(np.maximum(cosine_similarity(context_vec,sen1_vec),cosine_similarity(context_vec,sen2_vec),cosine_similarity(context_vec,sen3_vec)))

[[0.8021916]]


In [2]:
#import gensim
#import FastText as ft
model=FT_gensim.load_fasttext_format("/home/areej/anaconda3/AAAI-21-SDU-shared-task-2-AD-master/pretrained/wiki.en/wiki.en.bin") 

/home/areej/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [1]:
from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath


In [4]:
model['this']

/home/areej/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-1.99643850e-01,  8.51161256e-02, -5.51894903e-02,  1.31416783e-01,
       -3.42630148e-02,  1.16869882e-01,  2.42972206e-02, -3.65913302e-01,
       -1.60213172e-01,  1.30407602e-01, -5.11050969e-02, -1.94264352e-01,
        1.28881589e-01, -1.31380439e-01,  5.16473390e-02,  2.11300626e-02,
       -1.49447724e-01,  1.17996462e-01,  2.64452636e-01,  2.15311825e-01,
        5.97878359e-02,  1.45647615e-01, -1.35129586e-01, -8.24247524e-02,
       -3.03584695e-01,  1.36005461e-01, -1.56227082e-01,  7.74830207e-02,
        2.24251732e-01,  2.29710922e-01, -1.45234153e-01,  2.06172764e-01,
       -1.90676510e-01,  1.39753044e-01, -6.74275681e-02, -3.99615355e-02,
       -1.16826892e-02, -2.96144206e-02, -1.90305188e-01, -1.75481573e-01,
       -2.66916398e-02, -1.05777696e-01, -1.13316916e-01,  1.38660282e-01,
       -4.11015116e-02,  1.27849042e-01,  8.43051895e-02, -1.61175996e-01,
        2.20806375e-02, -3.68535183e-02, -5.00785224e-02, -3.79848480e-02,
        6.92265555e-02, -